In [ ]:
import uproot
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# fl = "/home/mart/auger/data/time_templates/MuTrans/OUTPUT_MuTransHistos_Xmax0440_40deg_MS_on.root"
# fl = "/home/mart/auger/MuTrans/OUTPUT_MuTransHistos_Xmax0500_50deg_MS_on.root"

def read_hist(fl):
    rootfile = uproot.open(fl)
    trpsi_hist = rootfile[rootfile.keys()[0]]
    hist = trpsi_hist.to_hist()
    h, tbins, lgr_bins, psi_bins = hist.to_numpy(flow=True)
    h = h[1:-1, 1:-1, 1:]
    t = (tbins[1:-2] + tbins[2:-1])/2
    lgr = (lgr_bins[1:-2] + lgr_bins[2:-1])/2
    psi_bins[-1] = 195
    psi = np.deg2rad((psi_bins[1:-1] + psi_bins[2:])/2)
#     h = h/h.max()
#     print(t.shape, lgr.shape, psi.shape, h.shape)
    #cannot interpolate in cospsi because then hist is not regular, or need to do complicated rebin. but lose magnetic field influence (which is turned off by default anyway)
    return h, t, lgr, psi

In [ ]:
from time_templates.templates.fit_muon_model_mean_traces import fit_Xmumax_lam
from time_templates.templates.muon_dsdt import MuondSdt

In [ ]:
theta = 40 #deg
Xmumax = 420
h, t, lgr, psi = read_hist(f"/home/mart/auger/data/MuTrans/OUTPUT_MuTransHistos_Xmax0{Xmumax}_{theta}deg_MS_on.root")

theta = np.deg2rad(theta)
ir = 50
ipsi = 10

r = 10**lgr[ir]
psi = psi[ipsi]
print(r, psi)
f, ax = plt.subplots(1, figsize=(10, 5))
dt = t[1]-t[0]
y = h[:, ir, ipsi]
yerr = np.sqrt(y)
# y /= norm
# yerr /= norm

muon_dsdt = MuondSdt(r, psi, theta)
muon_dsdt.use_kinematic_delay = 0

yerr[yerr <= 0] = 1
cdf = np.cumsum(y)
cdf /= cdf[-1]
t0 = np.interp(0., cdf, t)
t95 = np.interp(0.95, cdf, t)
mask = (t > t0) & (t < t95)
norm = y[mask].sum()*dt

from time_templates.utilities.fitting import plot_fit_curve

def func(t, Xmumax, lam):
    muon_dsdt.set_Xmumax_lambda(Xmumax)
    muon_dsdt.lam = lam
#     muon_dsdt.X0 = 0
    out = muon_dsdt.ds_dt(t, dndt=True)
    return norm*out

plot_fit_curve(t[mask], y[mask], yerr=yerr[mask], func=func, ax=ax, ebar_kws=dict(ls='', marker='.'), p0=[500, 40])
ax.legend()
# ax.set_xlim([0, 2000])

In [ ]:
norm

In [ ]:
y = h[:, ir, ipsi]
n = 10
plt.plot(t, y)
kernel = np.ones(n)/n
y_conv = np.convolve(y, kernel, mode='same')
plt.plot(t, y_conv)
plt.xlim([0, 1500])

In [ ]:
plt.plot(t, np.convolve(y, kernel, mode='full')[:len(t)])

In [ ]:
from scipy.interpolate import RegularGridInterpolator

In [ ]:
interp = RegularGridInterpolator((np.log10(t), lgr, psi), np.log10(h+1), bounds_error=False, fill_value=0)

In [ ]:
ir = 50
ipsi = 5
f, ax = plt.subplots(1, figsize=(10, 5))

ax.plot(t, h[:, ir, ipsi])
ax.plot(t, 10**interp((np.log10(t), lgr[ir], psi[ipsi]))-1, ls='--')
ax.set_xlim([0, 2000])

In [ ]:
for r in [500, 800, 1500]:
    for psi in [-np.pi, 0, np.pi/2]:
        func = lambda t: 10**interp((np.log10(t), np.log10(r), psi))-1
        plt.plot(func(np.arange(1, 3000, 25)))

In [ ]:
n = 3
for i in range(3):
    plt.plot(h[:, 50, n+i])
plt.xlim([0, 400])

In [ ]:
10**lgr